In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False
f_path = 'C:/Windows/Fonts/malgun.ttf' #윈도우
font_name = font_manager.FontProperties(fname=f_path).get_name()
rc('font',family=font_name)
%matplotlib inline
import warnings
warnings.simplefilter(action='ignore', category = FutureWarning)

In C:\Users\A\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In C:\Users\A\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\A\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The pgf.debug rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In C:\Users\A\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\A\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
train = pd.read_csv('../data/titanic/train.csv')
test = pd.read_csv('../data/titanic/test.csv')
#train = pd.concat([train.iloc[:,2:],test.iloc[:,1:]])
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Survived - 생존 여부 (0 = 사망, 1 = 생존)  
Pclass - 티켓 클래스 (1 = 1등석, 2 = 2등석, 3 = 3등석)  
Sex - 성별  
Age - 나이  
SibSp - 함께 탑승한 자녀 / 배우자 의 수  
Parch - 함께 탑승한 부모님 / 아이들 의 수  
Ticket - 티켓 번호  
Fare - 탑승 요금  
Cabin - 수하물 번호  
Embarked - 선착장 (C = Cherbourg, Q = Queenstown, S = Southampton)  

# Family

In [4]:
train['Family'] = 1 + train['SibSp'] + train['Parch']
test['Family'] = 1 + test['SibSp'] + test['Parch']

In [5]:
train['Sex'] = train['Sex'].apply(lambda x : 0 if x=='female' else (1 if x=='male' else x))
test['Sex'] = test['Sex'].apply(lambda x : 0 if x=='female' else (1 if x=='male' else x))

# Solo

In [7]:
train['Solo'] = (train['Family'] == 1)
test['Solo'] = (test['Family'] == 1)

In [9]:
train.loc[train['Solo'] == True,'Solo'] = 1
train.loc[train['Solo'] == False,'Solo'] = 0
test.loc[test['Solo'] == True,'Solo'] = 1
test.loc[test['Solo'] == False,'Solo'] = 0

In [10]:
conversion_nare = lambda x : x.split(',')[1].split('.')[0].strip()
train['Title'] = train['Name'].map(conversion_nare)
test['Title'] = test['Name'].map(conversion_nare)
train['Title'] = train['Title'].replace(['Capt', 'Col', 'Countess', 'Don','Dona', 'Dr', 'Jonkheer',
                                                 'Lady','Major', 'Rev', 'Sir','the Countess'], 'Other')
train['Title'] = train['Title'].replace('Mlle', 'Miss')
train['Title'] = train['Title'].replace('Mme', 'Mrs')
train['Title'] = train['Title'].replace('Ms', 'Miss')
test['Title'] = test['Title'].replace(['Capt', 'Col', 'Countess', 'Don','Dona', 'Dr', 'Jonkheer',
                                                 'Lady','Major', 'Rev', 'Sir','the Countess'], 'Other')
test['Title'] = test['Title'].replace('Mlle', 'Miss')
test['Title'] = test['Title'].replace('Mme', 'Mrs')
test['Title'] = test['Title'].replace('Ms', 'Miss')

In [11]:
train['Title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Other'], dtype=object)

In [12]:
train['TitleCode'] = train['Title'].apply(lambda x : 0 if x=='Mr' else (1 if x=='Mrs' else (2 if x=='MIss' else(3 if x=='Master' else 4))))
test['TitleCode'] = test['Title'].apply(lambda x : 0 if x=='Mr' else (1 if x=='Mrs' else (2 if x=='MIss' else(3 if x=='Master' else 4))))

# FamilyName

In [13]:
train['FamilyName'] = train['Name'].apply(lambda x : x.split(',')[0])
test['FamilyName'] = test['Name'].apply(lambda x : x.split(',')[0])

# Fare

In [15]:
test[test['Fare'].isnull()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family,Solo,Title,TitleCode,FamilyName
152,1044,3,"Storey, Mr. Thomas",1,60.5,0,0,3701,NaN,NaN,S,1,1,Mr,0,Storey


In [30]:
test[(test['Title'] == 'Mr') & (test['Pclass'] == 3) & (test['Solo'] == True)& (test['Age'] >30)]['Fare'].median()

7.875

In [31]:
test.loc[test['Fare'].isnull(),'Fare'] = 7.875

In [32]:
test[test.PassengerId == 1044]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family,Solo,Title,TitleCode,FamilyName
152,1044,3,"Storey, Mr. Thomas",1,60.5,0,0,3701,7.875,NaN,S,1,1,Mr,0,Storey


train['FareBin'] = pd.qcut(train['Fare'],5)
test['FareBin'] = pd.qcut(test['Fare'],5)

pd.qcut(train['Fare'],5)

In [38]:
train[train.Family==2].head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family,Solo,Title,TitleCode,FamilyName
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S,2,0,Mr,0,Braund
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C,2,0,Mrs,1,Cumings
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S,2,0,Mrs,1,Futrelle


In [43]:
train[train.Family==2].head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family,Solo,Title,TitleCode,FamilyName
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.25,NaN,S,2,0,Mr,0,Braund


In [ ]:
train[train.Family==2].head(1)

In [49]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    int64  
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
 12  Family       891 non-null    int64  
 13  Solo         891 non-null    object 
 14  Title        891 non-null    object 
 15  TitleCode    891 non-null    int64  
 16  FamilyName   891 non-null    object 
dtypes: float64(2), int64(8), object(7)
memory usage: 118.5+ KB


In [63]:
Fare_1=[]
for i in range(len(train)):
    if(train.Family[i]==1):
        Fare_1.append(train.Fare[i])
    elif(train.Family[i]==2):
        Fare_1.append(train.Fare[i]/2)
    elif(train.Family[i]==3):
        Fare_1.append(train.Fare[i]/3)
    elif(train.Family[i]==4):
        Fare_1.append(train.Fare[i]/4)
    elif(train.Family[i]==5):
        Fare_1.append(train.Fare[i]/5)
    elif(train.Family[i]==6):
        Fare_1.append(train.Fare[i]/6)
    elif(train.Family[i]==7):
        Fare_1.append(train.Fare[i]/7)
    elif(train.Family[i]==8):
        Fare_1.append(train.Fare[i]/8)
    elif(train.Family[i]==9):
        Fare_1.append(train.Fare[i]/9)
    elif(train.Family[i]==10):
        Fare_1.append(train.Fare[i]/10)
    else:
        Fare_1.append(train.Fare[i]/11)
train["Fare_1"]=Fare_1

In [65]:
Fare_1=[]
for i in range(len(test)):
    if(test.Family[i]==1):
        Fare_1.append(test.Fare[i])
    elif(test.Family[i]==2):
        Fare_1.append(test.Fare[i]/2)
    elif(test.Family[i]==3):
        Fare_1.append(test.Fare[i]/3)
    elif(test.Family[i]==4):
        Fare_1.append(test.Fare[i]/4)
    elif(test.Family[i]==5):
        Fare_1.append(test.Fare[i]/5)
    elif(test.Family[i]==6):
        Fare_1.append(test.Fare[i]/6)
    elif(test.Family[i]==7):
        Fare_1.append(test.Fare[i]/7)
    elif(test.Family[i]==8):
        Fare_1.append(test.Fare[i]/8)
    elif(test.Family[i]==9):
        Fare_1.append(test.Fare[i]/9)
    elif(test.Family[i]==10):
        Fare_1.append(test.Fare[i]/10)
    else:
        Fare_1.append(test.Fare[i]/11)
test["Fare_1"]=Fare_1

# Age

In [82]:
train.groupby('Title')['Age'].median()

Title
Master     3.5
Miss      21.0
Mr        30.0
Mrs       35.0
Other     48.5
Name: Age, dtype: float64

In [83]:
Age_map = dict(train.groupby('Title')['Age'].median())
Age_map

{'Master': 3.5, 'Miss': 21.0, 'Mr': 30.0, 'Mrs': 35.0, 'Other': 48.5}

In [84]:
#for k in Age_map.keys():
#    train.loc[(train.Title == k) & (train.Age.isnull()),'Age'] = Age_map[k]
train['Age'].fillna(train.groupby('Title')['Age'].transform('median'), inplace=True)

In [85]:
Age_map = dict(test.groupby('Title')['Age'].median())
Age_map

{'Master': 7.0, 'Miss': 22.0, 'Mr': 28.5, 'Mrs': 36.5, 'Other': 44.0}

In [86]:
#for k in Age_map.keys():
#    test.loc[(test.Title == k) & (test.Age.isnull()),'Age'] = Age_map[k]
test['Age'].fillna(test.groupby('Title')['Age'].transform('median'), inplace=True)

In [88]:
train.loc[train.Age < 10,'AgeCode'] = 0
train.loc[(train.Age >= 10) & (train.Age < 16),'AgeCode'] = 1
train.loc[(train.Age >= 16) & (train.Age < 20),'AgeCode'] = 2
train.loc[(train.Age >= 20) & (train.Age < 26),'AgeCode'] = 3
train.loc[(train.Age >= 26) & (train.Age < 30),'AgeCode'] = 4
train.loc[(train.Age >= 30) & (train.Age < 36),'AgeCode'] = 5
train.loc[(train.Age >= 36) & (train.Age < 40),'AgeCode'] = 6
train.loc[(train.Age >= 40) & (train.Age < 46),'AgeCode'] = 7
train.loc[(train.Age >= 46) & (train.Age < 50),'AgeCode'] = 8
train.loc[(train.Age >= 50) & (train.Age < 60),'AgeCode'] = 9
train.loc[train.Age >= 60,'AgeCode'] = 10

test.loc[test.Age < 10,'AgeCode'] = 0
test.loc[(test.Age >= 10) & (test.Age < 16),'AgeCode'] = 1
test.loc[(test.Age >= 16) & (test.Age < 20),'AgeCode'] = 2
test.loc[(test.Age >= 20) & (test.Age < 26),'AgeCode'] = 3
test.loc[(test.Age >= 26) & (test.Age < 30),'AgeCode'] = 4
test.loc[(test.Age >= 30) & (test.Age < 36),'AgeCode'] = 5
test.loc[(test.Age >= 36) & (test.Age < 40),'AgeCode'] = 6
test.loc[(test.Age >= 40) & (test.Age < 46),'AgeCode'] = 7
test.loc[(test.Age >= 46) & (test.Age < 50),'AgeCode'] = 8
test.loc[(test.Age >= 50) & (test.Age < 60),'AgeCode'] = 9
test.loc[test.Age >= 60,'AgeCode'] = 10

In [92]:
train_1=train[["Pclass","Sex","Age","Fare_1"]]
test_1=test[["Pclass","Sex","Age","Fare_1"]]

In [81]:
test_1.isnull().any()

Pclass    False
Sex       False
Age        True
Fare_1    False
dtype: bool

In [158]:
test_1

,Pclass,Sex,Age,Fare_1
0,3,1,34.5,7.829200
1,3,0,47.0,3.500000
2,2,1,62.0,9.687500
3,3,1,27.0,8.662500
4,3,0,22.0,4.095833
...,...,...,...,...
413,3,1,28.5,8.050000
414,1,0,39.0,108.900000
415,3,1,38.5,7.250000
416,3,1,28.5,8.050000


In [168]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=50,max_depth=6,random_state=0)
forest.fit(train_1,train[["Survived"]])
print("훈련 세트 정확도 : {:.2f}".format(forest.score(train_1,train[["Survived"]])))

훈련 세트 정확도 : 0.87


C:\Users\A\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [95]:
forest.feature_importances_

array([0.12582031, 0.46751473, 0.1749506 , 0.23171436])

In [169]:
forest_submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": forest.predict(test_1)
})
forest_submission.to_csv("../data/gender_submission_30.csv", index=False)

In [170]:
from sklearn.svm import SVC
svc=SVC(C=100).fit(train_1,train[["Survived"]])
print("훈련 세트 정확도 : {:.2f}".format(svc.score(train_1,train[["Survived"]])))
#0.76076

훈련 세트 정확도 : 0.79


C:\Users\A\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [171]:
forest_submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": svc.predict(test_1)
})
forest_submission.to_csv("../data/gender_submission_31.csv", index=False)

In [172]:
from sklearn.ensemble import GradientBoostingClassifier
gbrt = GradientBoostingClassifier(n_estimators=1000,random_state=0,max_depth=3,learning_rate=0.01)
gbrt.fit(train_1,train[["Survived"]])
print("훈련 세트 정확도 : {:.3f}".format(gbrt.score(train_1,train[["Survived"]])))
#0.72727

C:\Users\A\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


훈련 세트 정확도 : 0.882


In [173]:
forest_submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": gbrt.predict(test_1)
})
forest_submission.to_csv("../data/gender_submission_32.csv", index=False)

In [178]:
from sklearn.ensemble import ExtraTreesClassifier
extra_clf = ExtraTreesClassifier(n_estimators=100, max_leaf_nodes=100)
extra_clf.fit(train_1,train[["Survived"]])
print("훈련 세트 점수: {:.3f}".format(extra_clf.score(train_1,train[["Survived"]])))
#0.75598

훈련 세트 점수: 0.906


C:\Users\A\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [179]:
forest_submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": extra_clf.predict(test_1)
})
forest_submission.to_csv("../data/gender_submission_33.csv", index=False)

In [180]:
from sklearn.ensemble import ExtraTreesClassifier
extra_clf = ExtraTreesClassifier()
extra_clf.fit(train_1,train[["Survived"]])
print("훈련 세트 점수: {:.3f}".format(extra_clf.score(train_1,train[["Survived"]])))
#0.70813

훈련 세트 점수: 0.920


C:\Users\A\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [181]:
forest_submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": extra_clf.predict(test_1)
})
forest_submission.to_csv("../data/gender_submission_34.csv", index=False)
#

In [182]:
from sklearn.tree import  DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=1000, 
    algorithm="SAMME.R", learning_rate=1)
ada_clf.fit(train_1,train[["Survived"]])
print("훈련 세트 점수: {:.3f}".format(ada_clf.score(train_1,train[["Survived"]])))
#0.75119

C:\Users\A\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


훈련 세트 점수: 0.878


In [183]:
forest_submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": ada_clf.predict(test_1)
})
forest_submission.to_csv("../data/gender_submission_35.csv", index=False)

In [184]:
from sklearn.tree import  DecisionTreeClassifier
tree_clf=DecisionTreeClassifier(max_depth=5,random_state=3)
tree_clf.fit(train_1,train[["Survived"]])
print("훈련 세트 정확도 : {:.3f}".format(tree_clf.score(train_1,train[["Survived"]])))
#0.76555

훈련 세트 정확도 : 0.857


In [185]:
forest_submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": tree_clf.predict(test_1)
})
forest_submission.to_csv("../data/gender_submission_36.csv", index=False)

In [186]:
from sklearn.tree import  DecisionTreeClassifier
tree_clf=DecisionTreeClassifier(max_depth=10,random_state=3)
tree_clf.fit(train_1,train[["Survived"]])
print("훈련 세트 정확도 : {:.3f}".format(tree_clf.score(train_1,train[["Survived"]])))
#0.73684

훈련 세트 정확도 : 0.898


In [187]:
forest_submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": tree_clf.predict(test_1)
})
forest_submission.to_csv("../data/gender_submission_37.csv", index=False)

In [188]:
from sklearn.tree import  DecisionTreeClassifier
tree_clf=DecisionTreeClassifier(max_depth=1,random_state=3)
tree_clf.fit(train_1,train[["Survived"]])
print("훈련 세트 정확도 : {:.3f}".format(tree_clf.score(train_1,train[["Survived"]])))
#0.76555

훈련 세트 정확도 : 0.782


In [189]:
forest_submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": tree_clf.predict(test_1)
})
forest_submission.to_csv("../data/gender_submission_38.csv", index=False)

In [166]:
train.corr()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Family,TitleCode,Fare_1,AgeCode
PassengerId,1.000000,-0.005007,-0.035144,0.042939,0.043096,-0.057527,-0.001652,0.012658,-0.040143,-0.048858,0.026303,0.047035
Survived,-0.005007,1.000000,-0.338481,-0.543351,-0.078931,-0.035322,0.081629,0.257307,0.016639,0.422451,0.221600,-0.087406
Pclass,-0.035144,-0.338481,1.000000,0.131900,-0.354189,0.083081,0.018443,-0.549500,0.065997,-0.082202,-0.485079,-0.342093
Sex,0.042939,-0.543351,0.131900,1.000000,0.104921,-0.114631,-0.245489,-0.182333,-0.200988,-0.697246,-0.115143,0.125789
Age,0.043096,-0.078931,-0.354189,0.104921,1.000000,-0.265621,-0.188159,0.097947,-0.275544,-0.323094,0.150928,0.983216
SibSp,-0.057527,-0.035322,0.083081,-0.114631,-0.265621,1.000000,0.414838,0.159651,0.890712,0.213092,-0.094682,-0.263331
Parch,-0.001652,0.081629,0.018443,-0.245489,-0.188159,0.414838,1.000000,0.216225,0.783111,0.220506,-0.068978,-0.190942
Fare,0.012658,0.257307,-0.549500,-0.182333,0.097947,0.159651,0.216225,1.000000,0.217138,0.147996,0.840995,0.087784
Family,-0.040143,0.016639,0.065997,-0.200988,-0.275544,0.890712,0.783111,0.217138,1.000000,0.255802,-0.099173,-0.275369
TitleCode,-0.048858,0.422451,-0.082202,-0.697246,-0.323094,0.213092,0.220506,0.147996,0.255802,1.000000,0.098766,-0.328026


In [164]:
train[train['TitleCode'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family,Solo,Title,TitleCode,FamilyName,Fare_1,AgeCode


In [ ]:
PassengerId	Survived	Pclass	Sex	Age	SibSp	Parch	Fare	Family	TitleCode	Fare_1	AgeCode
-0.005007	1.000000	-0.338481	-0.543351	-0.078931	-0.035322	0.081629	0.257307	0.016639	0.422451	0.221600	-0.087406

In [ ]:
Pclass Sex TitleCode Fare_1

In [167]:
train_1=train[["Pclass","Sex","TitleCode","Fare_1"]]
test_1=test[["Pclass","Sex","TitleCode","Fare_1"]]